___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://miro.medium.com/max/1400/1*QJZ6W-Pck_W7RlIDwUIN9Q.jpeg" width="350px" height="180px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
#### <font color= #2E9AFE> `Martes y Viernes (Videoconferencia) de 13:00 - 15:00 hrs`</font>
- <Strong> Sara Eugenia Rodríguez </Strong>
- <Strong> Año </Strong>: 2023
- <Strong> Email: </Strong>  <font color="blue"> `cd682324@iteso.mx` </font>
___

<p style="text-align:right;"> Imagen recuperada de: https://miro.medium.com/max/1400/1*QJZ6W-Pck_W7RlIDwUIN9Q.jpeg</p>

### <font color= #2E9AFE> Tema: XGBoost - Regresión</font>

Los datos los podemos obtener de: https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

Columnas disponibles:
- AT: Atmospheric Temperature in C
- V: Exhaust Vacuum Speed
- AP: Atmospheric Pressure
- RH: Relative Humidity
- PE: Power Output

Power output (energía) es la variable a predecir

In [1]:
#Librerías
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Cargar datos
datos = pd.read_excel("Folds5x2_pp.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'Folds5x2_pp.xlsx'

In [ ]:
#vistazo de los datos


In [ ]:
#tamaño de los datos
datos.shape

In [ ]:
#tipo de datos


In [ ]:
#valores nulos?


In [ ]:
# Relación entre variable de salida y la temperatura
datos.plot(x ='AT', y = 'PE', kind ="scatter", 
                 figsize = [10,10],
                 color ="b", alpha = 0.3)
plt.title("Temperatura vs Energía")
plt.xlabel("Temperatura") 
plt.ylabel("Energía")
plt.show()

#Existe una correlación *negativa* entre la temperatura y la energía

In [ ]:
# Relación entre variable de salida y el Exhaust Vacuum Speed
datos.plot(x ='V', y = 'PE', kind ="scatter", 
                 figsize = [10,10],
                 color ="b", alpha = 0.3)
plt.title("Exhaust Vacuum Speed vs Energía")
plt.xlabel("Exhaust Vacuum Speed") 
plt.ylabel("Energía")
plt.show()

#Existe una correlación *negativa* entre la temperatura y el Exhaust Vacuum Speed

In [ ]:
# gráfico de correlacion
corr = datos.corr()
plt.figure(figsize = (9, 7))
sns.heatmap(corr, cmap="RdBu",
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

In [ ]:
#Separar nuestras X de las Y
X = datos.drop("PE", axis = 1).values
y = datos['PE'].values
y = y.reshape(-1, 1)

#Dividiendo los datos en prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                               test_size = 0.2, 
                                               random_state=42)



In [ ]:
#Construir el xgboost
#inicializar objeto de regresión
modelo = xgb.XGBRegressor(objective='reg:linear', seed=42)
#Entrenar modelo
modelo.fit(X_train, y_train)

In [ ]:
import sklearn.metrics as metrics
y_hat= modelo.predict(X_test)

#medidas de performance
r2 = metrics.r2_score(y_test,y_hat)
mse = metrics.mean_squared_error(y_test,y_hat)
print('R2:',r2)
print("MSE:",mse)

Ahora optimizamos parámetros con grid search y utilizamos cross validation

In [ ]:
#Parámetros para el grid search
gbm_param_grid = {
     'colsample_bytree': np.linspace(0.5, 0.9, 5), #porcentaje de variables a elegir cuando construyes tu arbol
     'n_estimators':[100, 200], #número de árboles a crear
     'max_depth': [10, 15, 20, 25] #profundidades
}
#Iniciar el modelo
gbm = xgb.XGBRegressor(objective='reg:linear', seed=42)

#Hacer el grid search
grid_mse = GridSearchCV(estimator = gbm, param_grid = gbm_param_grid, scoring = 'neg_mean_squared_error', cv = 5, verbose = 1)
#Ajustar a los datos de entrenamiento
grid_mse.fit(X_train, y_train)


In [ ]:
print("Mejores hiperparámetros encontrados: ",grid_mse.best_params_)

In [ ]:
#Construir y evaluar el XGBoost con los hiperparámetros óptimos
modelo_nuevo = xgb.XGBRegressor(objective='reg:linear',
                            seed=42,
                           max_depth=10,
                           colsample_bytree=0.8,
                            n_estimators=200)

#Entrenar modelo
modelo_nuevo.fit(X_train, y_train)

In [ ]:
#predecir
y_hat= modelo_nuevo.predict(X_test)

#metricas de performance
r2 = metrics.r2_score(y_test,y_hat)
mse = metrics.mean_squared_error(y_test,y_hat)
print('R2:',r2)
print("MSE:",mse)

**Ventajas**
- Modelo muy efectivo con bases de datos grandes y complejas
- No necesita escalamiento de variables
- Funciona bien con datos no lineales
- Modelo que muestra la importancia de las variables
- Tiene una herramienta interna para trabajar con datos nulos

**Desventajas**
- Modelo que tiende a sobreajustar (overfitting)
- Modelo de Caja negra -> no se puede interpretar
- Es más difícil tunear los hiperparámetros
- Modelo sensible a datos atípicos